# Library

In [5]:
import pymysql
import pandas as pd
import numpy as np

## DB 접속

In [59]:
# 전역변수 선언부
conn = None
cur = None

# 메인 코드
password = input('비밀번호를 입력하세요')
conn = pymysql.connect(host = 'localhost', port=3306, user='root', password=password, db='inflearn-bigquery', charset='utf8')
cur = conn.cursor() # 커서생성

## pokemon 테이블 불러오기

In [68]:
name = 'pokemon'
file_name = f'{name}.csv'
path = f'C:/Users/kbjoo/Downloads/초보자를 위한 BigQuery(SQL) 입문 강의 자료/dataset/{file_name}'
df = pd.read_csv(path)
df.shape

(251, 14)

In [69]:
df.head()

,id,kor_name,eng_name,type1,type2,total,hp,attack,defense,special_attack,special_defense,speed,generation,is_legendary
0,1,이상해씨,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,이상해풀,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,이상해꽃,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,4,파이리,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
4,5,리자드,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False


In [70]:
df.columns

Index(['id', 'kor_name', 'eng_name', 'type1', 'type2', 'total', 'hp', 'attack',
       'defense', 'special_attack', 'special_defense', 'speed', 'generation',
       'is_legendary'],
      dtype='object')

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               251 non-null    int64 
 1   kor_name         251 non-null    object
 2   eng_name         251 non-null    object
 3   type1            251 non-null    object
 4   type2            116 non-null    object
 5   total            251 non-null    int64 
 6   hp               251 non-null    int64 
 7   attack           251 non-null    int64 
 8   defense          251 non-null    int64 
 9   special_attack   251 non-null    int64 
 10  special_defense  251 non-null    int64 
 11  speed            251 non-null    int64 
 12  generation       251 non-null    int64 
 13  is_legendary     251 non-null    bool  
dtypes: bool(1), int64(9), object(4)
memory usage: 25.9+ KB


In [72]:
df.loc[df['type2'].isnull()]

,id,kor_name,eng_name,type1,type2,total,hp,attack,defense,special_attack,special_defense,speed,generation,is_legendary
3,4,파이리,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
4,5,리자드,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,7,꼬부기,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False
7,8,어니부기,Wartortle,Water,NaN,405,59,63,80,65,80,58,1,False
8,9,거북왕,Blastoise,Water,NaN,530,79,83,100,85,105,78,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,241,밀탱크,Miltank,Normal,NaN,490,95,80,105,40,70,100,2,False
241,242,해피너스,Blissey,Normal,NaN,540,255,10,10,75,135,55,2,False
242,243,라이코,Raikou,Electric,NaN,580,90,85,75,115,100,115,2,True
243,244,앤테이,Entei,Fire,NaN,580,115,115,85,90,75,100,2,True


In [73]:
# df['type2'] = df['type2'].fillna(np.nan)
df['type2'] = df['type2'].replace({np.nan:None})
# 이후 데이터베이스에 저장 시에는 자동으로 None이 NULL로 처리됨
df.iloc[3:9, :]

,id,kor_name,eng_name,type1,type2,total,hp,attack,defense,special_attack,special_defense,speed,generation,is_legendary
3,4,파이리,Charmander,Fire,None,309,39,52,43,60,50,65,1,False
4,5,리자드,Charmeleon,Fire,None,405,58,64,58,80,65,80,1,False
5,6,리자몽,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
6,7,꼬부기,Squirtle,Water,None,314,44,48,65,50,64,43,1,False
7,8,어니부기,Wartortle,Water,None,405,59,63,80,65,80,58,1,False
8,9,거북왕,Blastoise,Water,None,530,79,83,100,85,105,78,1,False


In [66]:
# 커서 생성
with conn.cursor() as cursor:
    # 3. 테이블에 데이터 삽입 (반복문을 통해 각 행 삽입)
    for index, row in df.iterrows():
        # 각 열 데이터를 적절히 매핑
        sql = """
        INSERT INTO pokemon (
            id, kor_name, eng_name, type1, type2, total, hp, attack, defense,
            special_attack, special_defense, speed, generation, is_legendary
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        # 각 데이터의 타입에 맞게 매핑
        cursor.execute(sql, (
            row['id'],                            # 숫자형
            row['kor_name'],                      # 문자형
            row['eng_name'],                      # 문자형
            row['type1'],                         # 문자형
            row['type2'],                         # 문자형, None으로 들어갈 수 있음
            int(row['total']),                    # 숫자형
            int(row['hp']),                       # 숫자형
            int(row['attack']),                   # 숫자형
            int(row['defense']),                  # 숫자형
            int(row['special_attack']),           # 숫자형
            int(row['special_defense']),          # 숫자형
            int(row['speed']),                    # 숫자형
            int(row['generation']),               # 숫자형
            bool(row['is_legendary'])             # boolean형
        ))

In [76]:
sql % (row['id'],                            # 숫자형
            row['kor_name'],                      # 문자형
            row['eng_name'],                      # 문자형
            row['type1'],                         # 문자형
            row['type2'],                         # 문자형, None으로 들어갈 수 있음
            int(row['total']),                    # 숫자형
            int(row['hp']),                       # 숫자형
            int(row['attack']),                   # 숫자형
            int(row['defense']),                  # 숫자형
            int(row['special_attack']),           # 숫자형
            int(row['special_defense']),          # 숫자형
            int(row['speed']),                    # 숫자형
            int(row['generation']),               # 숫자형
            bool(row['is_legendary']))

'\n        INSERT INTO pokemon (\n            id, kor_name, eng_name, type1, type2, total, hp, attack, defense,\n            special_attack, special_defense, speed, generation, is_legendary\n        ) VALUES (251, 세레비, Celebi, Psychic, Grass, 600, 100, 100, 100, 100, 100, 100, 2, False)\n        '

### GhatGPT가 알려준 방법
- 조건문 걸지 않아도 해결 됨.

In [ ]:
# # 'type2' 컬럼의 NaN 값을 None으로 변환
# df['type2'] = df['type2'].replace({np.nan: None})

# # NaN을 None으로 변환하여 삽입하는 부분 수정
# with conn.cursor() as cursor:
#     for index, row in df.iterrows():
#         sql = """
#         INSERT INTO pokemon (
#             id, kor_name, eng_name, type1, type2, total, hp, attack, defense,
#             special_attack, special_defense, speed, generation, is_legendary
#         ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#         """
        
#         # 각 데이터의 타입에 맞게 NaN을 None으로 변환
#         cursor.execute(sql, (
#             int(row['id']) if not np.isnan(row['id']) else None,
#             row['kor_name'],
#             row['eng_name'],
#             row['type1'],
#             row['type2'],  # type2가 NaN인 경우 None으로 처리
#             int(row['total']) if not np.isnan(row['total']) else None,
#             int(row['hp']) if not np.isnan(row['hp']) else None,
#             int(row['attack']) if not np.isnan(row['attack']) else None,
#             int(row['defense']) if not np.isnan(row['defense']) else None,
#             int(row['special_attack']) if not np.isnan(row['special_attack']) else None,
#             int(row['special_defense']) if not np.isnan(row['special_defense']) else None,
#             int(row['speed']) if not np.isnan(row['speed']) else None,
#             int(row['generation']) if not np.isnan(row['generation']) else None,
#             bool(row['is_legendary']) if not pd.isna(row['is_legendary']) else None
#         ))

In [67]:
# 커밋하여 변경 사항 적용
conn.commit()

# 연결 닫기
conn.close()

# End